In [1]:
import sempy.fabric as fabric

In [2]:
import notebookutils
import pandas as pd
import re
import requests
import msal
import json 

In [3]:
dataverse_prefix = "cr8dc"

## SEMPY

In [4]:
df_datasets = fabric.list_datasets()
df_datasets

,Dataset Name,Dataset ID,Created Timestamp,Last Update
0,Sales - What-if scenario (ENG),46d18c64-bf72-4b10-9b1c-e90ffffcb3ca,2024-12-16 09:15:33,NaT
1,lh_Dataverse,eaed42b2-d2d6-477b-9fbb-6cb07c6c0442,2021-02-12 23:00:58,NaT


In [5]:
execution_of_dax = fabric.evaluate_dax(
    workspace = 'e1ec8fb9-87a5-4a52-91d0-a1a60c1fbd31',
    dataset='46d18c64-bf72-4b10-9b1c-e90ffffcb3ca',
    dax_string="""
        EVALUATE 
            ADDCOLUMNS(
                SELECTCOLUMNS(
                    SUMMARIZE(
                        'fact Sales',
                        'dim Employees'[Employee],
                        'dim Pricelists'[Product name],
                        'dim DateKey'[Month]
                    ),
                    "Employee",'dim Employees'[Employee], 
                    "Product name",'dim Pricelists'[Product name], 
                    "Month",'dim DateKey'[Month]
                ),
                "Sold units", CALCULATE(SUM('fact Sales'[Quantity]))
            )
    """
)

execution_of_dax

,[Employee],[Product name],[Month],[Sold units]
0,Jan Novák,IoT sensor A,1,104
1,Tomáš Marný,IoT sensor A,1,411
2,Evžen Novotný,IoT sensor A,1,272
3,Kamil Kočka,IoT sensor A,1,89
4,Jürgen Nowak,IoT sensor A,1,280
...,...,...,...,...
1306,Emily Simon,Motherboard D,12,293
1307,John Che,Motherboard D,12,197
1308,James Allgood,Motherboard D,12,137
1309,Wei Jing,Motherboard D,12,158


## REST API

In [6]:
pbi_resource = "pbi"
pbi_Uri = 'https://api.powerbi.com/v1.0/myorg/'

In [7]:
wsId = 'e1ec8fb9-87a5-4a52-91d0-a1a60c1fbd31'
dsId= '46d18c64-bf72-4b10-9b1c-e90ffffcb3ca'

In [8]:
def get_token():
    return notebookutils.credentials.getToken(pbi_resource)

In [9]:
dax_query = """DEFINE
	VAR __DS0FilterTable = 
		FILTER(
			KEEPFILTERS(VALUES('Products calculation'[Products calculation])),
			'Products calculation'[Products calculation] = 3
		)
 
	VAR __DS0FilterTable2 = 
		FILTER(
			KEEPFILTERS(VALUES('Dynamic percentage'[Dynamic percentage])),
			'Dynamic percentage'[Dynamic percentage] = 40
		)
 
	VAR __DS0Core = 
		SUMMARIZECOLUMNS(
			'dim Employees'[Employee],
			__DS0FilterTable,
			__DS0FilterTable2,
			"Dynamic_vs_Total_Revenue_Difference", 'Measure'[# Dynamic vs Total Revenue Difference],
			"Total_Revenue", 'Measure'[# Total Revenue],
			"Dynamic_Revenue", 'Measure'[# Dynamic Revenue],
			"Dynamic_Profit", 'Measure'[# Dynamic Profit],
			"Total_Profit", 'Measure'[# Total Profit],
			"Dynamic_vs_Total_Profit_Difference", 'Measure'[# Dynamic vs Total Profit Difference],
			"Dynamic_Revenue_ratio", 'Measure'[% Dynamic Revenue ratio],
			"Dynamic_Profit_ratio", 'Measure'[% Dynamic Profit ratio]
		)
 
EVALUATE
	__DS0Core
"""

In [10]:
def post_ExecuteQuery(wsId, dsId, daxQ):
    fullurl = f'{pbi_Uri}/groups/{wsId}/datasets/{dsId}/executeQueries'
    pbi_access_token = get_token()
    headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {pbi_access_token}'}
    content = {"queries": [{"query": daxQ}],"serializerSettings": {"includeNulls": True}}
    response = requests.post(fullurl, json=content, headers = headers, timeout=60).json()['results'][0]['tables'][0]['rows']
    return response


In [11]:
def rename_columns_with_brackets(df,prefix):
    def extract_bracket_content(col_name):
        match = re.search(r'\[(.*?)\]', col_name)
        return (prefix+"_"+match.group(1)).lower() if match else col_name

    df.rename(columns=lambda x: extract_bracket_content(x), inplace=True)
    return df

In [12]:
response = post_ExecuteQuery(wsId,dsId,dax_query)
response = rename_columns_with_brackets(pd.DataFrame(response),dataverse_prefix)
display(response)

In [13]:
json_output = response.to_json(orient="records")
json_output = json.loads(json_output)

## Write to Dataverse

In [14]:
tenant_id = '08744604-44f5-451a-81b2-2b47618aee96'

In [15]:
keyvault_name = 'databrotherskeys' 
keyvault_ir = f'https://{keyvault_name}.vault.azure.net/'

client_id = notebookutils.credentials.getSecret(keyvault_ir,'dataverseFabricIntegrationClientId')
client_secret = notebookutils.credentials.getSecret(keyvault_ir,'dataverseFabricIntegrationClientSecret')
dataverse_name = 'org9b35785c' 
resource_uri = f'https://{dataverse_name}.crm4.dynamics.com'

authority_url = f'https://login.microsoftonline.com/{tenant_id}'
scope = [f'{resource_uri}/.default']

entity_name = f'{dataverse_prefix}_ingestedwhatifdatas'
entity_url = f'{resource_uri}/api/data/v9.2/{entity_name}'

In [16]:
app = msal.ConfidentialClientApplication(
    client_id,
    authority=authority_url,
    client_credential=client_secret
)

token_response = app.acquire_token_for_client(scopes=scope)
access_token = token_response.get('access_token')

In [17]:
# Data to be written to Dataverse
data = json_output

for row in json_output:
    
    token_response = app.acquire_token_for_client(scopes=scope)
    access_token = token_response.get('access_token')

    # Headers
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
        'OData-MaxVersion': '4.0',
        'OData-Version': '4.0'
    }

    # Write data to Dataverse
    response = requests.post(entity_url, headers=headers, json=row)

    if response.status_code == 204:
        print("Data written successfully!")
    else:
        print(f"Failed to write data. Status code: {response.status_code}")
        print(response.content)


Data written successfully!
Data written successfully!
Data written successfully!
Data written successfully!
Data written successfully!
Data written successfully!
Data written successfully!
Data written successfully!
Data written successfully!
Data written successfully!
Data written successfully!
Data written successfully!
Data written successfully!
Data written successfully!
Data written successfully!
Data written successfully!
